In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import pandas as pd
import eipy.ei as e
from eipy.additional_ensembles import MeanAggregation, CES

In [2]:
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [3]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True),
}

## data generation

In [4]:
import numpy as np

np.random.seed(38)

n_samples = 600

### single label data

In [5]:
# import numpy as np
# n_samples = 600

# n_time_steps = 8

# n_features = 30

# n_classes = 3 # 2 or 3


# if n_classes == 2:
#     # Generate toy dataset with regularity
#     X_class0 = np.random.randn(n_samples // 2, n_features, n_time_steps) + 1.5  # Add a bias to Class 0
#     X_class1 = np.random.randn(n_samples // 2, n_features, n_time_steps) - 1.5  # Subtract a bias from Class 1
#     X = np.concatenate([X_class0, X_class1])

#     # Generate labels (two classes)
#     y = np.concatenate([np.zeros(n_samples // 2), np.ones(n_samples // 2)])

# else:
#     # Generate toy dataset with less order and three classes
#     X_class0 = np.random.randn(n_samples // 3, n_features, n_time_steps) + 1.5  # Add a bias to Class 0
#     X_class1 = np.random.randn(n_samples // 3, n_features, n_time_steps) - 1.5  # Subtract a bias from Class 1
#     X_class2 = np.random.randn(n_samples // 3, n_features, n_time_steps) * 0.5  # Introduce disorder in Class 2
#     X = np.concatenate([X_class0, X_class1, X_class2])

#     # Generate labels (three classes)
#     y = np.concatenate([np.zeros(n_samples // 3), np.ones(n_samples // 3), 2 * np.ones(n_samples // 3)])

# # Shuffle the dataset
# shuffle_indices = np.random.permutation(n_samples)
# X = X[shuffle_indices]
# y = y[shuffle_indices]

### time dependent label data

In [6]:
import numpy as np

n_time_steps = 8
n_features = 30
n_classes = 3  # 2 or 3

if n_classes == 2:
    # Generate toy dataset with regularity
    X_class0 = np.random.randn(n_samples // 2, n_features, n_time_steps) + 1.5  # Add a bias to Class 0
    X_class1 = np.random.randn(n_samples // 2, n_features, n_time_steps) - 1.5  # Subtract a bias from Class 1
    X = np.concatenate([X_class0, X_class1])

    # Generate labels (two classes)
    y_class0 = np.zeros((n_samples // 2, n_time_steps))
    y_class1 = np.ones((n_samples // 2, n_time_steps))
    y = np.concatenate([y_class0, y_class1])

else:
    # Generate toy dataset with less order and three classes
    X_class0 = np.random.randn(n_samples // 3, n_features, n_time_steps) + 1.5  # Add a bias to Class 0
    X_class1 = np.random.randn(n_samples // 3, n_features, n_time_steps) - 1.5  # Subtract a bias from Class 1
    X_class2 = np.random.randn(n_samples // 3, n_features, n_time_steps) * 0.5  # Introduce disorder in Class 2
    X = np.concatenate([X_class0, X_class1, X_class2])

    # Generate labels (three classes)
    y_class0 = np.zeros((n_samples // 3, n_time_steps))
    y_class1 = np.ones((n_samples // 3, n_time_steps))
    y_class2 = 2 * np.ones((n_samples // 3, n_time_steps))
    y = np.concatenate([y_class0, y_class1, y_class2])

# Shuffle the dataset
shuffle_indices = np.random.permutation(n_samples)
X = X[shuffle_indices]
y = y[shuffle_indices]

## label arrays across time

In [7]:
# import numpy as np

# n_samples = 600
# n_time_steps = 8
# n_features = 30
# n_classes = 3

# # Generate toy dataset with three classes at every time step
# X_class0 = np.random.randn(n_samples // 3, n_features, n_time_steps) + 1.5  # Class 0
# X_class1 = np.random.randn(n_samples // 3, n_features, n_time_steps) - 1.5  # Class 1
# X_class2 = np.random.randn(n_samples // 3, n_features, n_time_steps) * 0.5   # Class 2

# # Introduce order across samples and time steps
# X = np.concatenate([X_class0, X_class1, X_class2])

# # Generate labels (three classes at every time step)
# y_class0 = np.zeros((n_samples // 3, n_time_steps))
# y_class1 = np.ones((n_samples // 3, n_time_steps))
# y_class2 = 2 * np.ones((n_samples // 3, n_time_steps))

# # Concatenate labels across classes
# y = np.concatenate([y_class0, y_class1, y_class2])

# # Shuffle the dataset
# shuffle_indices = np.random.permutation(n_samples)
# X = X[shuffle_indices]
# y = y[shuffle_indices]


## Training
BPs for every time point, train LSTM, gather outputs for classification

In [8]:
from sklearn.model_selection import train_test_split

meta_data = []
for t in range(n_time_steps):
    #time dependent data splitting
    labels_at_timestep = y[:, t]
    X_train, X_test, y_train, y_test = train_test_split(X, labels_at_timestep, test_size=0.25, random_state=38, stratify=labels_at_timestep)
    #modalities
    features_per_modality = int(n_features/3)
    X_train_dict = {f"Modality_{i}" : X_train[:,features_per_modality*i: features_per_modality*(i+1),:] for i in range(3)}
    X_test_dict = {f"Modality_{i}" : X_test[:,features_per_modality*i: features_per_modality*(i+1),:] for i in range(3)}
    X_train_timestep = {k: v[:,:,t] for k,v in X_train_dict.items()}

    EI_for_timestep = e.EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name=f"time step {t}",
                        model_building=False,
                        )
    print(f"generating metadata for timestep {t}")
    EI_for_timestep.fit_base(X_train_timestep, y_train)
    meta_data.append([EI_for_timestep.ensemble_training_data, EI_for_timestep.ensemble_test_data, EI_for_timestep.ensemble_training_data_final, EI_for_timestep.base_summary])

generating metadata for timestep 0
Training base predictors on Modality_0...
        
... for ensemble performance analysis...


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%


<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, ADAB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     0.0
1     0.0
2     1.0
3     0.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, DT), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, GB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%


<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, ADAB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     0.0
1     0.0
2     1.0
3     0.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, DT), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, GB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0


...

<class 'pandas.core.series.Series'>
0     1.0
1     0.0
2     1.0
3     1.0
4     1.0
     ... 
85    0.0
86    0.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, ADAB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, DT), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, GB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%


<class 'pandas.core.series.Series'>
0     1.0
1     0.0
2     1.0
3     1.0
4     1.0
     ... 
85    0.0
86    0.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, ADAB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, DT), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, GB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0


Generating ensemble training data: |██████████|100%
Generating ensemble test data: |██████████|100%


<class 'pandas.core.series.Series'>
0     1.0
1     0.0
2     1.0
3     1.0
4     1.0
     ... 
85    0.0
86    0.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, ADAB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, DT), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: labels, Length: 450, dtype: float64
<class 'pandas.core.series.Series'>
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0
89    2.0
Name: (Modality_0, GB), Length: 450, dtype: float64
0     2.0
1     0.0
2     1.0
3     2.0
4     1.0
     ... 
85    0.0
86    2.0
87    1.0
88    0.0


rearrange metadata so it's grouped by fold and then ordered by time steps

In [9]:
LSTM_training_data = [[dfs[0][i] for dfs in meta_data] for i in range(5)]
LSTM_test_data = [[dfs[1][i] for dfs in meta_data] for i in range(5)]
LSTM_training_data_final = [df[2] for df in meta_data]

In [10]:
for k in range(len(LSTM_training_data)):
    for t in range(len(LSTM_training_data[0])):
        print(LSTM_training_data[k][t].shape, t)

(360, 91) 0
(360, 91) 1
(360, 91) 2
(360, 91) 3
(360, 91) 4
(360, 91) 5
(360, 91) 6
(360, 91) 7
(360, 91) 0
(360, 91) 1
(360, 91) 2
(360, 91) 3
(360, 91) 4
(360, 91) 5
(360, 91) 6
(360, 91) 7
(360, 91) 0
(360, 91) 1
(360, 91) 2
(360, 91) 3
(360, 91) 4
(360, 91) 5
(360, 91) 6
(360, 91) 7
(360, 91) 0
(360, 91) 1
(360, 91) 2
(360, 91) 3
(360, 91) 4
(360, 91) 5
(360, 91) 6
(360, 91) 7
(360, 91) 0
(360, 91) 1
(360, 91) 2
(360, 91) 3
(360, 91) 4
(360, 91) 5
(360, 91) 6
(360, 91) 7


In [11]:
LSTM_training_data[0][0]

modality          Modality_0                                                  \
base predictor          ADAB                                   XGB             
sample                     0                                     0             
class                      0             1             2         0         1   
0               8.098635e-05  9.112715e-01  8.864756e-02  0.003859  0.930054   
1               9.999584e-01  8.180331e-07  4.076403e-05  0.983749  0.007839   
2               1.144542e-02  2.325767e-03  9.862288e-01  0.025257  0.008413   
3               9.999830e-01  6.914942e-08  1.688328e-05  0.997354  0.001521   
4               5.111852e-06  9.855346e-01  1.446025e-02  0.000993  0.992352   
..                       ...           ...           ...       ...       ...   
355             6.767345e-01  8.483312e-06  3.232570e-01  0.998177  0.000811   
356             3.921336e-10  1.000000e+00  1.682087e-08  0.003672  0.987904   
357             1.157764e-05  9.999417e-01  4.676613e-05  0.001483  0.997073   
358             8.214243e-11  1.000000e+00  5.124268e-09  0.000913  0.997794   
359             2.452624e-01  8.897681e-05  7.546487e-01  0.001029  0.000981   

modality                                       ...    Modality_2  \
base predictor             DT              RF  ...            NB   
sample                      0               0  ...             0   
class                  2    0    1    2     0  ...             0   
0               0.066087  0.0  1.0  0.0  0.01  ...  1.034073e-10   
1               0.008412  1.0  0.0  0.0  0.85  ...  1.000000e+00   
2               0.966330  0.0  0.0  1.0  0.16  ...  6.426491e-09   
3               0.001126  1.0  0.0  0.0  0.98  ...  1.000000e+00   
4               0.006655  0.0  1.0  0.0  0.00  ...  7.454367e-24   
..                   ...  ...  ...  ...   ...  ...           ...   
355             0.001012  1.0  0.0  0.0  1.00  ...  1.000000e+00   
356             0.008424  0.0  1.0  0.0  0.00  ...  2.941492e-13   
357             0.001444  0.0  1.0  0.0  0.01  ...  1.617503e-23   
358             0.001293  0.0  1.0  0.0  0.00  ...  6.274008e-22   
359             0.997990  0.0  0.0  1.0  0.12  ...  3.294553e-09   

modality                                                                      \
base predictor                                       MLP                       
sample                                                 0                       
class                      1             2             0         1         2   
0               9.999209e-01  7.914294e-05  1.198513e-03  0.640723  0.358078   
1               9.568518e-21  1.320064e-28  9.955606e-01  0.000053  0.004386   
2               2.968273e-07  9.999997e-01  6.855456e-03  0.012245  0.980900   
3               2.196306e-22  1.253562e-25  9.999430e-01  0.000002  0.000055   
4               1.000000e+00  7.972975e-26  9.668358e-07  0.999724  0.000275   
..                       ...           ...           ...       ...       ...   
355             1.539741e-15  1.644940e-15  9.824219e-01  0.000177  0.017401   
356             1.000000e+00  5.296132e-10  1.216637e-04  0.926422  0.073456   
357             1.000000e+00  2.862941e-24  7.596742e-07  0.999965  0.000034   
358             1.000000e+00  1.021580e-22  4.937875e-06  0.999579  0.000416   
359             1.233399e-06  9.999988e-01  4.584469e-03  0.022595  0.972821   

modality                                     labels  
base predictor       SVM                             
sample                 0                             
class                  0         1         2         
0               0.011892  0.659207  0.328901    1.0  
1               0.924415  0.058368  0.017217    0.0  
2               0.002287  0.007594  0.990119    2.0  
3               0.990773  0.004822  0.004405    0.0  
4               0.012896  0.980601  0.006503    1.0  
..                   ...       ...       ...    ...  
355             0.942111  0.017632  0

In [12]:
####HI OVER HERE###
import importlib
importlib.reload(e)
###HEYOOOOOO#

<module 'eipy.ei' from '/home/opc/eipy/eipy/ei.py'>

In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

performance_for_timesteps = []
for t in range(n_time_steps):
    model = Sequential()

    if n_classes == 2:
        model.add(LSTM(units=50, input_shape=(t+1,30))) # return_sequences=True to get prediction vectors at every time step
        model.add(Dense(units=1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    else:
        model.add(LSTM(units=50, input_shape=(t+1,30*n_classes))) # return_sequences=True to get prediction vectors at every time step
        model.add(Dense(units=3, activation='softmax'))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    ensembles = {"LSTM": model}

    ensemble_training_data = [x[:t+1] for x in LSTM_training_data]
    ensemble_test_data = [x[:t+1] for x in LSTM_test_data]
    # ensemble_training_data_final = [x[:t] for x in LSTM_training_data_final]
    LSTMs = e.EnsembleIntegration(
                base_predictors=base_predictors,
                k_outer=5,
                k_inner=5,
                n_samples=1,
                sampling_strategy=None,
                sampling_aggregation=None,
                n_jobs=-1,
                metrics=metrics,
                random_state=38,
                project_name="test",
                model_building=False,
                )
    LSTMs.modality_names = ["stuff"]
    LSTMs.ensemble_training_data = ensemble_training_data
    LSTMs.ensemble_test_data = ensemble_test_data
    # LSTMs.ensemble_training_data_final = ensemble_training_data_final

    LSTMs.fit_ensemble(ensemble_predictors=ensembles)
    performance_for_timesteps.append(LSTMs.ensemble_summary["metrics"])


2023-12-17 19:59:12.273995: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 19:59:12.301213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 19:59:12.301238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 19:59:12.302030: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 19:59:12.306595: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 19:59:12.307186: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/10
12/12 [==============================] - 1s 2ms/step - loss: 0.8017 - accuracy: 0.9389
Epoch 2/10
12/12 [==============================] - 0s 2ms/step - loss: 0.3581 - accuracy: 1.0000
Epoch 3/10
12/12 [==============================] - 0s 2ms/step - loss: 0.1392 - accuracy: 1.0000
Epoch 4/10
12/12 [==============================] - 0s 2ms/step - loss: 0.0597 - accuracy: 1.0000
Epoch 5/10
12/12 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 1.0000
Epoch 6/10
12/12 [==============================] - 0s 2ms/step - loss: 0.0206 - accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 0s 2ms/step - loss: 0.0151 - accuracy: 1.0000
Epoch 8/10
12/12 [==============================] - 0s 2ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 9/10
12/12 [==============================] - 0s 2ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 2ms/step
Epoch 1/10
12/12 [==============================] - 0s 

Analyzing ensembles: |██████████|100%

<class 'numpy.ndarray'>
[2, 0, 1, 2, 1, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 2, 0, 0, 1, 0, 0, 1, 2, 1, 0, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 0, 0, 1, 1, 1, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 1, 0, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 1, 2, 0, 2, 1, 2, 0, 0, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 0, 2, 2, 0, 2, 2, 1, 2, 1, 0, 2, 2, 0, 0, 2, 2, 1, 0, 1, 0, 0, 0, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 2, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, 2, 2, 1, 2, 1, 1, 0, 2, 1, 1, 1, 2, 2, 2, 0, 1, 1, 0, 1, 2, 2, 2, 0, 2, 1, 2, 2, 1, 2, 0, 2, 2, 1, 1, 0, 2, 0, 0, 2, 1, 2, 2, 2, 0, 2, 0, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 1, 2, 2, 2, 0, 1, 2, 0, 0, 0, 1, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2, 1, 0, 2, 0, 1, 2, 2, 1, 1, 0, 1, 1, 1, 1, 0, 2, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 2, 0, 2, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 1, 0, 1, 


Analyzing ensembles: |          |  0%

...

<class 'numpy.ndarray'>
[2, 0, 1, 2, 1, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 2, 0, 0, 1, 0, 0, 1, 2, 1, 0, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 0, 0, 1, 1, 1, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 1, 0, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 1, 2, 0, 2, 1, 2, 0, 0, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 0, 2, 2, 0, 2, 2, 1, 2, 1, 0, 2, 2, 0, 0, 2, 2, 1, 0, 1, 0, 0, 0, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 2, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, 2, 2, 1, 2, 1, 1, 0, 2, 1, 1, 1, 2, 2, 2, 0, 1, 1, 0, 1, 2, 2, 2, 0, 2, 1, 2, 2, 1, 2, 0, 2, 2, 1, 1, 0, 2, 0, 0, 2, 1, 2, 2, 2, 0, 2, 0, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 1, 2, 2, 2, 0, 1, 2, 0, 0, 0, 1, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2, 1, 0, 2, 0, 1, 2, 2, 1, 1, 0, 1, 1, 1, 1, 0, 2, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 2, 0, 2, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 1, 0, 1, 


Analyzing ensembles: |          |  0%

Epoch 1/10
12/12 [==============================] - 1s 3ms/step - loss: 0.4191 - accuracy: 0.9389
Epoch 2/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0388 - accuracy: 1.0000
Epoch 3/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 4/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 5/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 6/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 8/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 9/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 2ms/step
Epoch 1/10
12/12 [==============================] - 0s 

Analyzing ensembles: |██████████|100%

<class 'numpy.ndarray'>
[2, 0, 1, 2, 1, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 2, 0, 0, 1, 0, 0, 1, 2, 1, 0, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 0, 0, 1, 1, 1, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 1, 0, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 1, 2, 0, 2, 1, 2, 0, 0, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 0, 2, 2, 0, 2, 2, 1, 2, 1, 0, 2, 2, 0, 0, 2, 2, 1, 0, 1, 0, 0, 0, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 2, 2, 1, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, 2, 2, 1, 2, 1, 1, 0, 2, 1, 1, 1, 2, 2, 2, 0, 1, 1, 0, 1, 2, 2, 2, 0, 2, 1, 2, 2, 1, 2, 0, 2, 2, 1, 1, 0, 2, 0, 0, 2, 1, 2, 2, 2, 0, 2, 0, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 1, 2, 2, 2, 0, 1, 2, 0, 0, 0, 1, 0, 1, 0, 2, 0, 1, 2, 0, 0, 2, 1, 0, 2, 0, 1, 2, 2, 1, 1, 0, 1, 1, 1, 1, 0, 2, 0, 1, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 2, 0, 2, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 1, 0, 1, 

In [14]:
for t in range(n_time_steps):
    print(performance_for_timesteps[t])

           LSTM
precision   1.0
recall      1.0
f1          1.0
           LSTM
precision   1.0
recall      1.0
f1          1.0
           LSTM
precision   1.0
recall      1.0
f1          1.0
           LSTM
precision   1.0
recall      1.0
f1          1.0
           LSTM
precision   1.0
recall      1.0
f1          1.0
           LSTM
precision   1.0
recall      1.0
f1          1.0
           LSTM
precision   1.0
recall      1.0
f1          1.0
           LSTM
precision   1.0
recall      1.0
f1          1.0
